# Summary

Today, we're going to calculate rates of convergence for our
nonparametric regression estimators. In the current homework, we're
tackling a few cases with a pen-and-paper approach. Today, we're going
to do it computationally for Monotone Regression and Bounded Variation Regression. 
If you like, you should be able to replicate the approach to tackle lipschitz regression,
convex regression, sobolev regression, or anything else we've implemented. 
Width calculation code just isn't that different from least squares code, so
it's easy to swap in new models we've already got regression code for.


In [ ]:
suppressPackageStartupMessages({
library(tidyverse)
library(CVXR)
})
CVXR::add_to_solver_blacklist('OSQP')

lab.theme = theme(plot.background = element_rect(fill = "transparent", colour = NA),
		    					panel.background = element_rect(fill = "transparent", colour = NA),
                  legend.background = element_rect(fill="transparent", colour = NA),
                  legend.box.background = element_rect(fill="transparent", colour = NA),
                  legend.key = element_rect(fill="transparent", colour = NA),
				panel.grid.major=element_line(color=rgb(0,0,0,.05,   maxColorValue=1)),
	      panel.grid.minor=element_line(color=rgb(0,0,0,.02,   maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))
theme_set(lab.theme)

## Review

Recall what we've shown about the relationship between error bounds and
the gaussian width of neighborhoods in our model. Letting $\mu^\star$ be
the best approximation to $\mu$ in our model $\mathcal{M}$,

$$
\begin{aligned}
\lVert \hat \mu - \mu^\star \rVert_{L_2(P_n)} < s + 2\sqrt{\frac{2\{1+2\log(2n)\}}{\delta n}} &\ \ \ \text{ w.p. \ $1-\delta$ \ if } \ \ \  s^2 \ge 2\sigma \text{w}(\mathcal{M}_s) \\
\text{where} & \ \mathcal{M}_s = \left\{ m \in \mathcal{M}  \ : \ \lVert m - \mu^\star \rVert_{L_2(P_n)} \le s \right\} \\ 
\text{and}   & \ \text{w}(\mathcal{V}) = \operatorname{E} \max_{v \in \mathcal{V}}\frac{1}{n}\sum_{i=1}^n g_i v_i \quad \text{ for a gaussian vector } \ g \sim N(0, I). 
\end{aligned}
$$



## Essential Approach

The reason we're bothering with gaussian width at all is that linear
combinations of gaussians concentrate around their mean, so with high
probability, gaussian width and 'sample gaussian width' are close.

$$ 
\hat{\text{w}}(\mathcal{V}) := \max_{v \in \mathcal{V}}\frac{1}{n}\sum_{i=1}^n g_i v_i \quad \text{ satisfies } \quad \hat{\text{w}}(\mathcal{V}) \approx \text{w}(\mathcal{V}). 
$$

This means that if we want to find a radius $s$ satisfying our
inequality above, we can more or less just find a solution to the
inequality $s^2 \ge 2 \sigma \hat{\text{w}}(\mathcal{M}_s)$. And
because we want the best error bound we can get, we want the smallest
$s$ satisfying this. Or something close to it.

If we can maximize the linear function $l(v) = g^T v$ over vectors in
the set $\mathcal{M}_s$ efficiently, it's easy to do this. We make
a list of values of $s$ to try, sample a gaussian vector
$g \sim N(0,1)$, calculate
$\hat{\text{w}}(\mathcal{M}_s)=\max_{v \in \mathcal{M}_s} g^T v/n$
at each $s$ in our list, and choose the smallest for which
$s^2 \ge 2\sigma \hat{\text{w}}(\mathcal{M}_s)$. To visualize
what's going on, we can plot the curves $f(s)=s^2$ and
$g(s)=2\sigma \hat{\text{w}}(\mathcal{M}_s)$ and look for the
point $s$ at which they cross. We saw one of these plots at the end of
our last lecture and today you'll be making your own.

**Caveat**. $\hat{\text{w}}(\mathcal{V}) \approx \text{w}(\mathcal{V})$ in large samples essentially because that's when sample averages are close to expected values. To get a better estimate of $\text{w}(\mathcal{V})$ in small samples, we can average multiple copies of $\hat{\text{w}}(\mathcal{V})$, i.e. copies we get for multiple noise vectors $g$.

# A First Pass

## Calculating Width

We're going to do this for monotone regression. This will take less work
than you might think. Let's compare what the function `monotonereg` we
wrote a while back does to what we need to do now.

$$ 
\hat\mu = \operatorname*{argmin}_{\text{increasing} \ m } \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2  \quad \text{ vs. } \quad 
  \hat{\text{w}}(\mathcal{M}_s) = \max_{\substack{\text{increasing} \ m \\ \lVert m \rVert_{L_2(P_n)} \le s}} g^T m / n 
$$

There are basically three differences. 

1. We're maximizing a linear function of $m$ instead of minimizing a quadratic one. 
2. We're interested in the *value* of the maximum instead of the *argument* at
which it occurs. 
3. We have an additional constraint on the sample
two-norm of our curve $m$. 

If you we make these changes to the monotone
regression code we've been using, we're good. Here's what that looks like. 

In [ ]:
invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

monotone           = function(m,X) { diff(m) >= 0 }
bounded.variation  = function(m,X) { sum(abs(diff(m))) <= 1 }
neighborhood.width = function(in.model, X, mustarX, s, g) {
    # find the unique elements of X and the inverse mapping
    unique.X = invert.unique(X)
    uX = X[unique.X$elements]
    n = length(X)
  
    # Step 1.
    # We tell CVXR we're thinking about a vector of unknowns m in R^p.
    # and permute and duplicate these into a vector mX with n elements in correspondence with X_1 ... X_n 
    m = Variable(length(unique.X$elements))
    mX = m[unique.X$inverse]

    inner.product = sum(g * (mX-mustarX)) / n
    neighborhood.constraint = sum((mX-mustarX)^2)/n <= s^2
    model.constraint = in.model(m, uX) 
    constraints = list(model.constraint, neighborhood.constraint)

    # solve and return the maximal value
    solved = solve(Problem(Maximize(inner.product), constraints))
    the.max = solved$value
    the.argmax = solved$getValue(mX)
    attr(the.max, 'argmax') = the.argmax 
    the.max
}

The code below will calculate $\text{w}(\mathcal{M}_s)$ of neighborhoods of $\mu_\star$ in both of our models.
It'll do it for a number of radii $s$ and samples sizes $n$. 

This'll take a couple minutes to run. It takes about 8 minutes on my laptop.

In [ ]:
set.seed(2)
sigma=1/4
ns = c(2,25,50,100,200)
radii=seq(0,1,length.out=40)
mustar = function(x) { x } 
reps = 10
models = list(bv=bounded.variation, monotone=monotone)


X=runif(max(ns))
g=sigma*rnorm(max(ns)*reps)
dim(g) = c(max(ns), reps)
grid = expand.grid(n=ns, s=radii, rep=1:reps, model=names(models))

widths = grid |> pmap(function(n,s,rep,model) { 
  model = models[[model]]
  model |> neighborhood.width(X[1:n], mustar(X[1:n]), s, g[1:n,rep])
})

# Visualizing Width

To visualize the width calculation, you can run this code. 

In the background, you'll see:

- The functions in the model, i.e. the pairs $\{x=m(X_1), y=m(X_2)\}$ for $m \in \mathcal{M}$, will be shown in blue. 
- The set of functions (pairs) within a distance $s$ of $\mu_\star$, will be shown in red.
- The neighborhood $\mathcal{M}_s$ will be where the red and blue regions overlap.

In the foreground, you'll see:

- The vector $2\varepsilon$ in orange.
- The function $m$ maximizing $\langle \varepsilon, m-\mu_\star \rangle$ in blue.
- A breakdown of $2\varepsilon$ into components parallel and perpendicular to $m$.
  - The orange dot on the dashed blue line indicates the parallel component.
  - The dashed orange line shows the perpendicular component.

There are sets of these arrows corresponding to two draws of our random vector $\varepsilon$.

In [ ]:
x = seq(-10,10,length.out=1000)
projection = function(epsilon, dm) { 
  epsilon.proj = dm * sum(dm*epsilon)/sum(dm^2)
  epsilon.proj
}
model_geom = function(model) { 
  if(model == 'monotone') {
    geom_ribbon(aes(x=x, ymin=x, ymax=Inf), 
                fill='blue', alpha=.1, color='lightgray')
  } else { # it's bv
    geom_ribbon(aes(x=x, ymin=x-1, ymax=x+1), 
                fill='blue', alpha=.1, color='lightgray')
  }
}
plot.neighborhood.width = function(model, mustarX, argmaxes, gs, s) {
  background = ggplot() + 
    model_geom(model) + 
    geom_point(aes(x=mustarX[1], y=mustarX[2]), color='black', alpha=.3) + 
    geom_polygon(aes(x=mustarX[1]+s*cos(pi*x), y=mustarX[2]+s*sin(pi*x)), 
                 fill='red', color='lightgray', alpha=.1, linewidth=.2)

    arrow.style = arrow(length = unit(0.2, "cm"), ends='last', type='closed')
    all.arrows = 1:ncol(argmaxes) |> map(function(rr) { 
      argmax = argmaxes[,rr]
      g = 2*gs[,rr]  
      pg = projection(g, argmax-mustarX) 

      arrows = list(
        annotate('segment',
          x=mustarX[1], xend=mustarX[1]+g[1], 
          y=mustarX[2], yend=mustarX[2]+g[2], 
          arrow = arrow.style, color='orange', linewidth=1, alpha=.5),
        annotate('segment', 
          x=mustarX[1], xend=argmax[1], 
          y=mustarX[2], yend=argmax[2], 
          arrow = arrow.style, color='blue', linewidth=1, alpha=.5),
        annotate('line',
          x=mustarX[1] + x*(argmax[1]-mustarX[1]),
          y=mustarX[2] + x*(argmax[2]-mustarX[2]),
          color='blue', linetype='dashed', linewidth=.4, alpha=.5),
        annotate('segment', 
          x=mustarX[1]+g[1], xend=mustarX[1] + pg[1], 
          y=mustarX[2]+g[2], yend=mustarX[2] + pg[2], 
          color='orange', linetype='dashed', linewidth=.4, alpha=.5),
        annotate('point', 
          x=mustarX[1] + pg[1], 
          y=mustarX[2] + pg[2], 
          color='orange')
      )
    }) |> list_c()
    background + all.arrows + labs(x='', y='') + ggtitle(model)
}


plot.width.at.radius = function(model, s) { 
  ws = which(grid$n==2 & grid$s==s & grid$model==model)
  argmaxes = do.call(cbind, map(widths[ws], function(w) { attr(w, 'argmax') }))
  reps = grid[ws,]$rep
  rr = c(3,6)
  coords = coord_cartesian(xlim=c(-2,2), ylim=c(-2,2))
  plot.neighborhood.width(model, mustar(X)[1:2], 
                          argmaxes[1:2,rr, drop=FALSE],
                          g[1:2,reps[rr],drop=FALSE], s*sqrt(2)) + coords  
}

plot.radii = radii[seq(2, length(radii), by=4)]
for(model in  names(models)) { 
  for(s in plot.radii) { 
    print(model |> plot.width.at.radius(s))
  }
}

# Crossing Points and Error Bounds

To visualize radius $s$ at which $s^2 \ge 2\sigma\text{w}(\mathcal{M}_s)$ and how it varies with sample size, you can run this.
For each model and sample size $n$, it'll plot ...

  - $f(s)=s^2$ in blue.
  - $g(s)=2\sigma\text{w}(\mathcal{M}_s)$ in red.


The point $s_\star$ at which these curves cross is the smallest $s$ for which $s^2 \ge 2\sigma\text{w}(\mathcal{M}_s)$.
That's the best choice of $s$ to plug into our bound formula $\lVert \hat\mu - \mu_\star \rVert \le s + \sqrt{\frac{2\{1+2\log(2n)\}}{\delta n}}$.

In [ ]:
delta = .8

crossing.plot = grid |>
  add_column(w=unlist(widths)) |>
  filter(grid$n > 2) |>
  ggplot() +
    geom_point(aes(x=s, y=2*sigma*w), alpha=.2, size=.5, color='red') + 
    stat_summary(aes(x=s, y=2*sigma*w), fun=mean, geom='line', color='red') +
    geom_line(aes(x=s, y=s^2), color='blue') + 
    facet_grid(rows=vars(n), cols=vars(model))
crossing.plot + coord_cartesian(xlim=c(0,.15), ylim=c(0,.025))

What are these radii?  We can roughly calculate the crossing point by 
looking for the smallest radius $s$ for which $s^2 \ge 2\sigma\text{w}(\mathcal{M}_s)$
*in the set of radii we've tried*. Having done this, we can plot the resulting probability $1-\delta$ error bound $s+2\sigma\sqrt{\frac{2\{1+2\log(2n)\}}{\delta n }}$ as a function of $n$.
I've used $\delta=.2$.

In [ ]:
summaries = grid |>
  add_column(w=unlist(widths)) |>
  group_by(n,s,model) |>
  summarize(w=mean(w), .groups='drop')

best.s = summaries |>
  group_by(n,model) |>
  summarize(s = min(s[s^2 >= 2*sigma*w]), .groups='drop')

efron.stein = function(n,delta=.2) { sqrt(2*(1+2*log(2*n))/(delta*n)) }
ggplot(best.s) +
  geom_point(aes(x=n, y=s+2*sigma*efron.stein(n), color=model)) +
  geom_line(aes(x=n,  y=s+2*sigma*efron.stein(n), color=model))

If you're curious, you can compare these bounds to the errors we see empirically, 
i.e. to error curves like the ones we drew in the convergence rates lab and subsequent homeworks. 
As a quick approximation to doing this, I'll estimate the rate $s \propto n^{-b}$ at which these bounds
decrease to zero. For both models, we get something like $n^{-1/3}$, like we did for the empirical errors earlier in the semester.

In [ ]:
for(modelname in names(models)) { 
  rate.model = nls(formula = s ~ a*n^(-b), 
                   start=list(a=1, b=1/2), 
                   data=best.s |> filter(model==modelname & n > 2))
  print(model)
  print(summary(rate.model))
}

## The Cost of Simplifying our Crossing Condition 

The bound we've been using, stated below, isn't quite the one we proved in lecture. 
$$
\lVert \hat \mu - \mu \rVert \le s + 2\sigma \sqrt{\frac{2\{1+2\log(2n)\}}{\delta n}} 
\quad \text{ for } \quad s^2 \ge 2\sigma\text{w}(\mathcal{M}_s).
$$
What we did prove is this.
$$ 
\lVert \hat\mu - \mu \rVert \le \tilde s \quad \text{ for } \quad \tilde s^2 \ge  \textcolor{blue}{2\sigma\left\{\text{w}(\mathcal{M}_{\tilde s}) + \tilde{s} \sqrt{\frac{2\{1+2\log(2n)\}}{\delta n}}\right\}}.
$$
That the second implies the first is a homework question. In the 'crossing plot' below, I've added a dashed red line indicating 
the $\textcolor{blue}{\text{right side of the latter inequality}}$. I've also added a purple dashed line where $\sqrt{\frac{2\log(1/\delta)}{n}}$ replaces the square root factor on the right side. This is what we'd be working with if we'd used the [Borell-TIS](https://en.wikipedia.org/wiki/Borell–TIS_inequality) inequality, which is specific to gaussian noise and a bit harder to prove, in place of Efron-Stein.^[For a quick survey of inequalities like this, see [this post](https://terrytao.wordpress.com/2010/01/03/254a-notes-1-concentration-of-measure/) on Terry Tao's blog. The Borell-TIS inequality is a consequence of what Tao calls the
*Gaussian concentration inequality for Lipschitz functions*. For a very detailed survey, see Boucheron, Lugosi, and Massart's book *Concentration inequalities: A nonasymptotic theory of independence*.] 

We can see that the crossing point $\tilde s$ for the second bound tends to be well to the right of the one for $s$.
The shift is, however, much smaller when we use the sharper Borell-TIS inequality in place of Efron-Stein.

In [ ]:
borell.tis  = function(n,delta=.2) { sqrt(2*log(1/delta)/n) }

crossing.plot + 
  stat_summary(aes(x=s, y=2*sigma*(w+s*efron.stein(n))) , 
    fun=mean, geom='line', linetype='dashed', color='red') +
  stat_summary(aes(x=s, y=2*sigma*(w+s*borell.tis(n))), 
    fun=mean, geom='line', linetype='dashed', color='purple')

How do the $\tilde s$ bounds compare to the 'additive' $s + \ldots$ bounds? Let's plot them as a function of $n$. What we see is that the $\tilde s$ bounds tend to be substantially better.  

**Something to think about.**  We use the 'additive version' of our bound because it makes things simpler. It lets us think about the impact of the model $\mathcal{M}$ we're using and the probability $\delta$ with which we tolerate out bound being invalid as two separate things. Is the cost we're paying necessary to break things down like this? Or did we not derive the best possible 'additive version', or more generally the best possible version that lets us break things down like this, in our homework exercise? If you get somewhere with this,
let me know.

In [ ]:
quietmin = function(x) { min(c(Inf, x)) }
best.s = summaries |>
  group_by(n,model) |>
  summarize(efron.stein = quietmin(s[s^2 >= 2*sigma*(w+s*efron.stein(n))]),
            borell.tis  = quietmin(s[s^2 >= 2*sigma*(w+s*borell.tis(n))]),
            expected    = quietmin(s[s^2 >= 2*sigma*w]),
            .groups = 'drop') |>
  mutate(additive.efron.stein = s + 2*sigma*s*efron.stein(n,delta),
         additive.borell.tis  = s + 2*sigma*s*borell.tis(n,delta))

In [ ]:
best.s |> pivot_longer(cols = c('expected', 'efron.stein', 'borell.tis', 'additive.efron.stein', 'additive.borell.tis'),
                       names_to = 'type', values_to = 's') |>
          filter(n > 2 & model == 'monotone' & is.finite(s)) |>
  ggplot() +
    geom_point(aes(x=n, y=s, color=type)) + 
    geom_line(aes(x=n, y=s, color=type))